In [ ]:
import matplotlib.pyplot as plt
import mne
import seaborn as sns
import torch
from braindecode import EEGClassifier
from sklearn.pipeline import make_pipeline
from skorch.callbacks import EarlyStopping, EpochScoring
from skorch.dataset import ValidSplit
import matplotlib.pyplot as plt
import pandas as pd
import os
from torch.utils.data import Dataset, DataLoader, BatchSampler
from torch.utils.data._utils.collate import default_collate
from sklearn.pipeline import Pipeline
from torch.utils.data import SequentialSampler


from moabb.datasets import BNCI2014_001, BNCI2014_004
from moabb.evaluations import CrossSessionEvaluation
from moabb.paradigms import MotorImagery
from moabb.utils import setup_seed
import numpy as np
from moabb.evaluations import AllRunsEvaluationModified, AllRunsEvaluation, AllRunsEvaluationSubjectSpecific
from shallow import CollapsedShallowNet, SubjectDicionaryFCNet, SubjectOneHotNet, SubjectOneHotConvNet, SubjectDicionaryConvNet, SubjectOneHotConvNet2, SubjectAdvIndexFCNet,ShallowPrivateTemporalDictNet, ShallowPrivateTemporalDictNetSlow


In [ ]:
class SubjectDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, subjects):
        self.X = X.get_data() if isinstance(X, mne.BaseEpochs) else X
        self.y = y
        self.subjects = np.array(subjects)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        X_i = self.X[idx].astype('float32')
        y_i = self.y[idx]
        subject_i = self.subjects[idx]
        return {'x': X_i, 'subject_ids': subject_i}, y_i


class SubjectBatchSampler(torch.utils.data.Sampler):
    def __init__(self, subjects, batch_size, shuffle=True):
        self.subjects = np.array(subjects)
        self.batch_size = batch_size
        self.shuffle = shuffle

        # Build a dict of subject -> indices
        self.subject_indices = {}
        for subject in np.unique(self.subjects):
            indices = np.where(self.subjects == subject)[0]
            if self.shuffle:
                np.random.shuffle(indices)
            self.subject_indices[subject] = indices

        self.subject_list = list(self.subject_indices.keys())
        if self.shuffle:
            np.random.shuffle(self.subject_list)

    def __iter__(self):
        for subject in self.subject_list:
            indices = self.subject_indices[subject]
            # Split indices into batches
            batches = [indices[i:i + self.batch_size] for i in range(0, len(indices), self.batch_size)]
            batches = [batch for batch in batches if len(batch) > 0]  # Remove empty batches
            if self.shuffle:
                np.random.shuffle(batches)
            for batch in batches:
                yield batch

    def __len__(self):
        total_batches = sum(
            (len(indices) + self.batch_size - 1) // self.batch_size
            for indices in self.subject_indices.values()
        )
        return total_batches


def collate_fn(batch):
    X_list, y_list = zip(*batch)
    x_list = [item['x'] for item in X_list]
    subject_ids_list = [item['subject_ids'] for item in X_list]
    X_tensor = torch.stack([torch.tensor(sample) for sample in x_list])
    y_tensor = torch.tensor(y_list)
    subject_ids_tensor = torch.tensor(subject_ids_list)
    return {'x': X_tensor, 'subject_ids': subject_ids_tensor}, y_tensor




class EEGClassifierSubjectBatches(EEGClassifier):
    def __init__(self, *args, per_subject_batches=False, **kwargs):
        super().__init__(*args, **kwargs)
        self.per_subject_batches = per_subject_batches
        self.subjects_ = None  # To be set in fit method

    def fit(self, X, y, subjects=None, **fit_params):
        if self.per_subject_batches:
            if subjects is None:
                raise ValueError("Subjects must be provided when per_subject_batches is True.")
            self.subjects_ = subjects
        return super().fit(X, y, **fit_params)

    def get_dataset(self, X, y=None):
        if self.per_subject_batches and y is not None:
            return SubjectDataset(X, y, self.subjects_)
        else:
            return super().get_dataset(X, y)

    def get_iterator(self, dataset, training=False):
        if self.per_subject_batches:
            if isinstance(dataset, torch.utils.data.Subset):
                indices = dataset.indices
                adjusted_subjects = [self.subjects_[i] for i in indices]
            else:
                adjusted_subjects = self.subjects_

            batch_sampler = SubjectBatchSampler(
                subjects=adjusted_subjects,
                batch_size=self.batch_size,
                shuffle=training  # Shuffle during training, not during validation
            )

            return DataLoader(
                dataset,
                batch_sampler=batch_sampler,
                collate_fn=collate_fn,
            )
        else:
            return super().get_iterator(dataset, training)





In [ ]:

mne.set_log_level(False)

# Print Information PyTorch
print(f"Torch Version: {torch.__version__}")

# Set up GPU if it is there
cuda = torch.cuda.is_available()
device = "cuda" if cuda else "cpu"
print("GPU is", "AVAILABLE" if cuda else "NOT AVAILABLE")

seed = 3
setup_seed(seed)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Hyperparameter

# learning rate 1e-4

# batch = 2^7
LEARNING_RATE = 0.0001  # parameter taken from Braindecode
WEIGHT_DECAY = 0  # parameter taken from Braindecode
BATCH_SIZE = 128  # parameter taken from BrainDecode
EPOCH = 20 #3000
PATIENCE = 100
fmin = 4
fmax = 100
tmin = 0
tmax = None


dataset = BNCI2014_001()
paradigm = MotorImagery(
    fmin=fmin, fmax=fmax, tmin=tmin, tmax=tmax
)

X, _, _ = paradigm.get_data(dataset=dataset)

subjects = dataset.subject_list
        

# def make_classifier(module):
#     clf = EEGClassifier(
#         module=module,  
#         module__n_chans=X.shape[1],  # number of input channels
#         module__n_outputs=len(dataset.event_id),  # number of output classes
#         module__n_times=X.shape[2],  # length of the input signal in time points
#         optimizer=torch.optim.Adam,
#         optimizer__lr=LEARNING_RATE,
#         batch_size=BATCH_SIZE,
#         max_epochs=EPOCH,
#         train_split=ValidSplit(0.2, random_state=seed),
#         device=device,
#         callbacks=[
#             EarlyStopping(monitor="valid_loss", patience=PATIENCE),
#             EpochScoring(
#                 scoring="accuracy", on_train=True, name="train_acc", lower_is_better=False
#             ),
#             EpochScoring(
#                 scoring="accuracy", on_train=False, name="valid_acc", lower_is_better=False
#             ),
#         ],
#         verbose=1,
#     )
    
#     return clf
def make_classifier(module, per_subject_batches=False):
    clf = EEGClassifierSubjectBatches(
        module=module,
        module__n_chans=X.shape[1],
        module__n_outputs=len(dataset.event_id),
        module__n_times=X.shape[2],
        optimizer=torch.optim.Adam,
        optimizer__lr=LEARNING_RATE,
        batch_size=BATCH_SIZE,
        max_epochs=EPOCH,
        train_split=ValidSplit(0.2, random_state=seed),
        device=device,
        callbacks=[
            EarlyStopping(monitor="valid_loss", patience=PATIENCE),
            EpochScoring(
                scoring="accuracy", on_train=True, name="train_acc", lower_is_better=False
            ),
            EpochScoring(
                scoring="accuracy", on_train=False, name="valid_acc", lower_is_better=False
            ),
        ],
        verbose=1,
        per_subject_batches=per_subject_batches,
    )
    return Pipeline([
        ('classifier', clf),
    ])


clf = make_classifier(SubjectOneHotNet)

clf2 = make_classifier(SubjectDicionaryFCNet)

clf3 = make_classifier(CollapsedShallowNet, per_subject_batches=True)

clf4 = make_classifier(SubjectOneHotConvNet)

clf5 = make_classifier(SubjectOneHotConvNet2)

clf6 = make_classifier(SubjectDicionaryConvNet)

clf7 = make_classifier(SubjectAdvIndexFCNet)

clf8 = make_classifier(ShallowPrivateTemporalDictNet, per_subject_batches=True)



pipes = {
    "CollapsedShallowNet": make_classifier(ShallowPrivateTemporalDictNetSlow, per_subject_batches=True),
    # Other pipelines...
}
#pipes = {"SubjectOneHotConvNet": make_pipeline(clf4), "CollapsedShallowNet": make_pipeline(clf3), "SubjectDicionaryFCNet": make_pipeline(clf2), "SubjectOneHotNet": make_pipeline(clf),}
#pipes = {"CollapsedShallowNet": make_pipeline(clf3), "SubjectDicionaryConvNet": make_pipeline(clf6),}
#one with them all
#pipes = {"SubjectOneHotNet": make_pipeline(clf), "SubjectDicionaryFCNet": make_pipeline(clf2), "CollapsedShallowNet": make_pipeline(clf3), "SubjectOneHotConvNet": make_pipeline(clf4), "SubjectOneHotConvNet2": make_pipeline(clf5), "SubjectDicionaryConvNet": make_pipeline(clf6), "SubjectAdvIndexFCNet": make_pipeline(clf7),}
results_list = []
# Ensure the output directory exists
output_dir = "./results"
os.makedirs(output_dir, exist_ok=True)

# Modify plot and data saving within the loop
for pipe_name, pipe in pipes.items():
    unique_suffix = f"{pipe_name}_braindecode_example"
    
    evaluation = AllRunsEvaluationSubjectSpecific(
        paradigm=paradigm,
        datasets=[dataset],
        suffix=unique_suffix,
        overwrite=True,
        return_epochs=True,
        random_state=seed,
        n_jobs=1,
        hdf5_path=f"{output_dir}/{pipe_name}",
        save_model=True
    )
    
    # Run the evaluation process for this pipeline
    results = evaluation.process({pipe_name: pipe})

    # Save results to CSV
    results_df = pd.DataFrame(results)
    results_df.to_csv(f"{output_dir}/{pipe_name}_results.csv", index=False)

    # Save individual bar plot
    plt.figure(figsize=(10, 6))
    sns.barplot(data=results_df, y="score", x="subject", palette="viridis")
    plt.title(f"Model Performance by Subject - {pipe_name}")
    plt.ylabel("Score")
    plt.xlabel("Subject")
    plt.savefig(f"{output_dir}/{pipe_name}_performance.png")
    plt.close()
    
    results_list.append(results_df)

# Concatenate all results
results_all = pd.concat(results_list)
results_all.to_csv(f"{output_dir}/all_results.csv", index=False)

# Save combined bar plot
plt.figure(figsize=(12, 6))
sns.barplot(data=results_all, x="subject", y="score", hue="pipeline", palette="viridis")
plt.title("Scores per Subject for Each Pipeline")
plt.xlabel("Subject")
plt.ylabel("Score")
plt.legend(title="Pipeline")
plt.savefig(f"{output_dir}/combined_performance.png")
plt.close()

Choosing from all possible events


Torch Version: 2.2.2
GPU is NOT AVAILABLE
We try to set the tensorflow seeds, but it seems that tensorflow is not installed. Please refer to `https://www.tensorflow.org/` to install if you need to use this deep learning module.


/Users/kasperhenningsen/Desktop/moabb---Private-Encoder-Thesis-/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 48 events (all good), 2 – 6 s (baseline off), ~8.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12
 'feet': 12
 'tongue': 12>
  warn(f"warnEpochs {epochs}")
/Users/kasperhenningsen/Desktop/moabb---Private-Encoder-Thesis-/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 48 events (all good), 2 – 6 s (baseline off), ~8.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12
 'feet': 12
 'tongue': 12>
  warn(f"warnEpochs {epochs}")
/Users/kasperhenningsen/Desktop/moabb---Private-Encoder-Thesis-/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 48 events (all good), 2 – 6 s (baseline off), ~8.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12
 'feet': 12
 'tongue': 12>
  warn(f"warnEpochs {epochs}")
/Users/kasperhenningsen/Desktop/moabb---Private-Encoder-Thesis-/moabb/datasets/preprocessing.py:279: UserWarning

Creating SubjectDataset with 4140 samples.


BNCI2014-001-AllRuns:   0%|          | 0/1 [00:23<?, ?it/s]

Error: More than one subject ID detected in the batch


TypeError: cross_entropy_loss(): argument 'input' (position 1) must be Tensor, not NoneType